In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import matplotlib
matplotlib.rc('font', size=18)

import pandas as pd
import xarray as xr
from scipy.stats import linregress
from tqdm.notebook import tqdm

import sys
sys.path.append('../')

from general_purpose import cartopy_plots as cplt

In [ ]:
ts = xr.open_dataset('HEAVY--Land_and_Ocean_LatLong1.nc')

In [ ]:
ts

In [ ]:
LON,LAT = np.meshgrid(ts.longitude.data,ts.latitude.data)

## Plot the land_mask

In [ ]:
cplt.mfp(LON, LAT, ts.land_mask.data.reshape(*ts.land_mask.shape,1),
         figsize=(12,6),
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=['Annual mean temperature [C]'],
        )

In [ ]:
ocean_mask = ts.land_mask == 0
atlantic_mask = ocean_mask.copy()

# mask out everything below 35 deg south
atlantic_mask[atlantic_mask.latitude < -35] = False
atlantic_mask[atlantic_mask.latitude > 80] = False

atlantic_mask[:,atlantic_mask.longitude > 25] = False
atlantic_mask[:,atlantic_mask.longitude < -120] = False

for i in range(atlantic_mask.shape[0]):
    if atlantic_mask.latitude[i] > 20 or atlantic_mask.latitude[i] < -35:
        continue
    j = np.argmax(atlantic_mask.data[i])
    while atlantic_mask.data[i,j]:
        j += 1
    atlantic_mask[i,:j] = False
    
atlantic_mask[:, atlantic_mask.longitude < -100] = False

plt.close(2)
fig = plt.figure(num=2, figsize=(12,9))
ax = fig.add_subplot(111, projection=cplt.ccrs.PlateCarree())

cplt.geo_plotter(ax, LON, LAT,
                 atlantic_mask,
                 greenwich=True,
                 draw_gridlines=False, draw_labels=False,
                 put_colorbar=True
                )

## Plot the climatology

In [ ]:
ts.climatology.shape

In [ ]:
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
ofl = (4,3)
ims = cplt.mfp(LON, LAT, ts.climatology.data.transpose(1,2,0),
               figsize=(16,9),
               one_fig_layout=ofl,
               # draw_coastlines=False,
               projections=cplt.ccrs.PlateCarree(),
               colorbar='shared',
               put_colorbar=False,
               extents=None,
               titles=months,
              )

plt.close(3)
fig,ax = plt.subplots(num=3, figsize=(10,2))
plt.colorbar(ims[0],cax=ax,orientation='horizontal', extend='both', label='T [C]')
fig.tight_layout()

### Compute climatology anomaly

In [ ]:
mean = ts.climatology.mean('month_number')

In [ ]:
cplt.mfp(LON, LAT, mean.data.reshape(*mean.shape,1),
         figsize=(12,6),
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=['Annual mean temperature [C]'],
         draw_labels=True,
         draw_gridlines=True
        )

In [ ]:
ano_clim = ts.climatology - mean

In [ ]:
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
ofl = (4,3)
cplt.mfp(LON, LAT, ano_clim.data.transpose(1,2,0),
         figsize=(16,9),
         one_fig_layout=ofl,
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='shared',
         extents=None,
         titles=months,
        )

## Plot a few snapshots

In [ ]:
ts.temperature.shape

In [ ]:
i = 12*100
ofl = (4,3)
ims = cplt.mfp(LON, LAT, ts.temperature.data[i:i+np.prod(ofl)].transpose(1,2,0),
               figsize=(16,9),
               one_fig_layout=ofl,
               # draw_coastlines=False,
               projections=cplt.ccrs.PlateCarree(),
               colorbar='shared',
               put_colorbar=False,
               extents=None,
               titles=months,
              )

plt.close(3)
fig,ax = plt.subplots(num=3, figsize=(10,2))
plt.colorbar(ims[0],cax=ax,orientation='horizontal', extend='both', label='T [C]')
fig.tight_layout()

In [ ]:
plt.close(19)
plt.figure(figsize=(9,6), num=19)
ts.temperature.isel(latitude=125, longitude=200).plot()
plt.plot(ts.time.data, ts.temperature.isel(longitude=200, latitude=125).data - ano_clim_periodic[:,125,200])
fig.tight_layout()

In [ ]:
pn = ts.temperature.isnull().mean(['longitude','latitude'])

In [ ]:
pno = ts.temperature.where(ocean_mask).isnull().mean(['longitude','latitude']) - (1 - np.mean(ocean_mask))

In [ ]:
pna = ts.temperature.where(atlantic_mask).isnull().mean(['longitude','latitude']) - (1 - np.mean(atlantic_mask))

In [ ]:
plt.close(4)
fig,ax = plt.subplots(figsize=(9,6), num=4)
pn.plot(label='all')
pno.plot(label='ocean only')
pna.plot(label='altantic only')
plt.ylabel('Fraction of missing points')
plt.xlabel('Year')
plt.legend()

### Plot a few time series

In [ ]:
lon = 30
lat = 35
ilon = np.argmin((ts.longitude.data - lon)**2)
ilat = np.argmin((ts.latitude.data - lat)**2)
lon = ts.longitude.data[ilon]
lat = ts.latitude.data[ilat]

plt.close(9)
fig,ax = plt.subplots(num=9, figsize=(9,6))
ts.temperature.isel(longitude=ilon, latitude=ilat).plot()

plt.axhline(0, linestyle='dashed', color='black')

fig.tight_layout()

## Global mean temperature dataset

### Make the file readable by pandas

In [ ]:
with open('HEAVY--global_temp-sea_ice_from_air.txt', 'r') as orig:
    with open('HEAVY--global_temp-sea_ice_from_air.csv', 'w') as dest:
        for line in orig.readlines():
            if line.startswith('%'):
                line = f'#{line[1:]}'
            else:
                line = ','.join(filter(lambda x: x != '', line.split(' ')))
            dest.write(line)

### Load the data

In [ ]:
df = pd.read_csv('HEAVY--global_temp-sea_ice_from_air.csv', sep=',', comment='#')
df['time'] = df['year'] + (df['month'] - 0.5)/12

In [ ]:
df

In [ ]:
plt.close(9)
fig,ax = plt.subplots(num=9, figsize=(9,6))
plt.plot(df['time'], df['a-m'])

plt.xlabel('time [yr]')
plt.ylabel('Global mean temperature anomaly [C]')

plt.axhline(0, linestyle='dashed', color='black')

fig.tight_layout()

In [ ]:
lon = 65
lat = 35
ilon = np.argmin((ts.longitude.data - lon)**2)
ilat = np.argmin((ts.latitude.data - lat)**2)
lon = ts.longitude.data[ilon]
lat = ts.latitude.data[ilat]

plt.close(10)
fig,ax = plt.subplots(num=10, figsize=(9,6))
plt.scatter(df['a-m'], ts.temperature.isel(longitude=ilon, latitude=ilat).data, alpha=0.5)

fig.tight_layout()

## Detrend the data

In [ ]:
def regress(t):
    mask = np.isnan(t.data)
    res = linregress(df['a-m'][~mask], t[~mask])
    return res.slope, res.intercept

In [ ]:
res = xr.apply_ufunc(regress, ts.temperature, input_core_dims=[['time']], output_core_dims=[[],[]], vectorize=True)

In [ ]:
reg = xr.Dataset({'slope': res[0], 'intercept': res[1]})
reg

In [ ]:
reg.to_netcdf('HEAVY--regression-tALL.nc')

In [ ]:
cplt.mfp(LON, LAT, np.stack([reg['slope'].data,reg['intercept'].data], axis=-1),
         figsize=(12,12),
         one_fig_layout=(2,1),
         # draw_coastlines=False,
         projections=cplt.ccrs.PlateCarree(),
         colorbar='individual',
         extents=None,
         titles=['Slope', 'Intercept [C]'],
         draw_labels=True,
         draw_gridlines=True
        )